In [1]:
import pandas as pd
import airbnb_scraper.pyairbnb as pyairbnb
import json
import time
from tqdm import tqdm
from datetime import datetime

In [ ]:

listings_path = r"C:\Users\fraxi\OneDrive\Desktop\code task\Amsterdam\listings.csv"
output_path = r"C:\Users\fraxi\OneDrive\Desktop\code task\Amsterdam\transaction_pairs.csv"
top_n = 10
language = "en"
proxy_url = None


In [ ]:


df = pd.read_csv(listings_path)
cutoff_date = datetime(2017, 1, 1)
df['first_review'] = pd.to_datetime(df['first_review'], errors='coerce')
df = df[df['first_review'] < cutoff_date]


df = df[['id', 'host_id']].dropna().drop_duplicates()


all_pairs = []


In [ ]:



for idx, row in tqdm(df.iterrows(), total=len(df), desc="Scraping listings"):
    room_id = int(row['id'])
    host_id = str(int(row['host_id']))
    room_url = f"https://www.airbnb.com/rooms/{room_id}"

    try:

        details_data, price_input, cookies = pyairbnb.get_metadata_from_url(room_url, language, proxy_url)
        product_id = price_input['product_id']
        api_key = price_input['api_key']


        raw_reviews = pyairbnb.reviews.get(api_key, product_id, "USD", language, proxy_url)

        for review in raw_reviews:
            guest = review.get("reviewer", {})
            host_response = review.get("response", None)
            created_at_str = review.get("createdAt")
            if not guest or not host_response or not created_at_str:
                continue


            pair = {
                "listing_id": room_id,
                "listing_url": room_url,
                "host_id": host_id,
                "guest_id": guest.get("id"),
                "guest_name": guest.get("firstName"),
                "guest_comment": review.get("comments"),
                "host_response": host_response,
                "rating": review.get("rating"),
                "date": created_at_str,
                "language": review.get("language", ""),
                "guest_count": review.get("guestCount", None),
            }
            all_pairs.append(pair)

    except Exception as e:
        print(f"romm {room_id} error: {e}")
        continue

    time.sleep(1)  



Scraping listings:   0%|          | 5/1442 [00:30<1:47:43,  4.50s/it]


❌ 房源 47061 报错: 'NoneType' object is not subscriptable


Scraping listings:   8%|▊         | 112/1442 [08:33<1:35:58,  4.33s/it]


❌ 房源 693675 报错: 'NoneType' object is not subscriptable


Scraping listings:   9%|▉         | 129/1442 [09:42<1:06:08,  3.02s/it]

...

Scraping listings:  90%|████████▉ | 1291/1442 [1:21:11<11:37,  4.62s/it]


❌ 房源 13952479 报错: HTTP Error 503: 


Scraping listings:  91%|█████████ | 1308/1442 [1:22:17<05:44,  2.57s/it]


❌ 房源 14134576 报错: 'NoneType' object is not subscriptable


Scraping listings: 100%|██████████| 1442/1442 [1:30:27<00:00,  3.76s/it]


In [5]:
# ====== 保存为 CSV ======
df_out = pd.DataFrame(all_pairs)
df_out.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"\n✅ 共保存 {len(df_out)} 条 2017 年之前的有效交易对到 {output_path}")



✅ 共保存 14011 条 2017 年之前的有效交易对到 C:\Users\fraxi\OneDrive\Desktop\code task\Amsterdam\transaction_pairs.csv


In [7]:
#读取交易对 保留2017年之前的评论

df_out['date'] = pd.to_datetime(df_out['date'], errors='coerce')
df_out = df_out[df_out['date'] < '2017-01-01']

unique_guest_ids = df_out['guest_id'].dropna().unique()
print(f"共有 {len(unique_guest_ids)} 个唯一的 guest_id")

共有 3440 个唯一的 guest_id
